CNN made to compare to the pretrained transfer model. Has piss poor accuracy because dataset is only a few thousand images.

In [1]:
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.vgg19 import VGG19

In [2]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)
data_dir = pathlib.Path(archive).with_suffix('')

In [3]:
batch_size = 32
img_height = 180
img_width = 180


train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
  

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [6]:
train_ds.class_names

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [14]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [15]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [16]:
aug_ds = train_ds.map(
  lambda x, y: (resize_and_rescale(x, training=True), y))

In [17]:
model = Sequential()

model.add(Conv2D(16, 3, activation='relu', padding='same', input_shape=(180, 180, 3)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(32, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())



model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

In [18]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 90, 90, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 45, 45, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 22, 22, 64)      

In [19]:
model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(train_ds,
  validation_data=val_ds,
  epochs=5
)

Epoch 1/5
92/92 [==============================] - 15s 156ms/step - loss: 0.7144 - accuracy: 0.7197 - val_loss: 1.0898 - val_accuracy: 0.6158
Epoch 2/5
92/92 [==============================] - 15s 157ms/step - loss: 0.6279 - accuracy: 0.7565 - val_loss: 2.2452 - val_accuracy: 0.4850
Epoch 3/5
92/92 [==============================] - 14s 153ms/step - loss: 0.5618 - accuracy: 0.7742 - val_loss: 1.0483 - val_accuracy: 0.6458
Epoch 4/5
92/92 [==============================] - 15s 163ms/step - loss: 0.5223 - accuracy: 0.7878 - val_loss: 1.3829 - val_accuracy: 0.6403
Epoch 5/5
92/92 [==============================] - 15s 158ms/step - loss: 0.5436 - accuracy: 0.7885 - val_loss: 1.3836 - val_accuracy: 0.6390


In [24]:
loss, acc = model.evaluate(val_ds)
print("Accuracy", acc)

23/23 [==============================] - 1s 33ms/step - loss: 1.3836 - accuracy: 0.6390
Accuracy 0.6389645934104919
